# Test experimental model

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path
import time

import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import numpy as np

import shutil
from pprint import pprint

import model
import evaluator
import plotting

import neuron
import LFPy

import neuroplotlib as nplt

%matplotlib notebook
# %matplotlib inline


In [ ]:
# import logging
# logging.basicConfig(level=logging.DEBUG)

In [ ]:
probe_type = "linear"
model_name = "experimental" #'hallermann' # "hay" hay_ais
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
# compile model-specific mechanisms
if not (model_folder / "x86_64").is_dir():
    curr_dir = Path(".").absolute()
    os.chdir(model_folder)
    print(os.getcwd())
    os.system("nrnivmodl mechanisms")
    os.chdir(curr_dir)
else:
    print(f"Mechanisms already compiled for {model_name}")
    compiled_folder = Path("./x86_64")
    if compiled_folder.is_dir():
        shutil.rmtree(compiled_folder)
    shutil.copytree(model_folder / "x86_64", compiled_folder)

In [ ]:
morphology = "../data/experimental/cell1_210301/morphology/morphology_corrected.swc"
probe_file = "../data/experimental/cell1_210301/extracellular/probe.json"
params_release = "experimental_model/parameters_release_test.json"

In [ ]:
cell = model.create_experimental_model(morphology_file=morphology,
                                       parameters_file=params_release)

In [ ]:
param_names = [param.name for param in cell.params.values() if not param.frozen]
probe = model.define_electrode(probe_file=probe_file)

fitness_protocols = evaluator.define_protocols(
    model_name,
    feature_set="soma",
    feature_file=f"{model_name}_model/features.json",
    electrode=probe,
    protocols_with_lfp=["firepattern_120"]
)

fitness_calculator = ephys.objectivescalculators.ObjectivesCalculator([])

sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe)

eva = ephys.evaluators.CellEvaluator(                    
        cell_model=cell,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)

In [ ]:
test_protocol = fitness_protocols["APWaveform_260"]

In [ ]:
response = test_protocol.run(cell, param_values={}, sim=sim)

In [ ]:
plt.figure()
plt.plot(response["APWaveform_260.soma.v"]["time"], response["APWaveform_260.soma.v"]["voltage"])

### Check morphology and probe

In [ ]:
cell.instantiate(sim=sim)

In [ ]:
fig, ax = plt.subplots()
ax.plot(probe.positions[:, 0], probe.positions[:, 1], marker="s", ls="", color="y", alpha=0.2)
nplt.plot_detailed_neuron(cell.LFPyCell, ax=ax, plane="xy")

In [ ]:
fig, ax = plt.subplots()
ax.plot(probe.positions[:, 0], probe.positions[:, 2], marker="s", ls="", color="y", alpha=0.2)
nplt.plot_detailed_neuron(cell.LFPyCell, ax=ax, plane="xz")
ax.axis("off")